<a href="https://colab.research.google.com/github/Mariehf64/fev23cda_french_industry/blob/Dominique_1_Exploration/Exploration_Base_etablissements_par_tranche_effectifs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd  # appel des bibliohthéques
import numpy as np
import matplotlib.pyplot as plt
##ceci est un test###

In [75]:
# appel du fichier source  => base_etablissement_par_tranche_effectif.csv
source=pd.read_csv('/content/base_etablissement_par_tranche_effectif.csv')

# création d'une copie pour éviter toute corruption du fichier source
df=source.copy() 

In [76]:
# prémière visusalisation 
df.head()

,CODGEO,LIBGEO,REG,DEP,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500
0,01001,L'Abergement-Clémenciat,82,01,25,22,1,2,0,0,0,0,0,0
1,01002,L'Abergement-de-Varey,82,01,10,9,1,0,0,0,0,0,0,0
2,01004,Ambérieu-en-Bugey,82,01,996,577,272,63,46,24,9,3,2,0
3,01005,Ambérieux-en-Dombes,82,01,99,73,20,3,1,2,0,0,0,0
4,01006,Ambléon,82,01,4,4,0,0,0,0,0,0,0,0


In [72]:
# premières analyses 


df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36681 entries, 0 to 36680
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CODGEO    36681 non-null  object
 1   LIBGEO    36681 non-null  object
 2   REG       36681 non-null  int64 
 3   DEP       36681 non-null  object
 4   E14TST    36681 non-null  int64 
 5   E14TS0ND  36681 non-null  int64 
 6   E14TS1    36681 non-null  int64 
 7   E14TS6    36681 non-null  int64 
 8   E14TS10   36681 non-null  int64 
 9   E14TS20   36681 non-null  int64 
 10  E14TS50   36681 non-null  int64 
 11  E14TS100  36681 non-null  int64 
 12  E14TS200  36681 non-null  int64 
 13  E14TS500  36681 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 3.9+ MB


**DESCRIPTION DES VARIABLES**

CODGEO : ID géographique de la ville<br>
LIBGEO : nom de la ville<br>
REG : numéro de région<br>
DEP : numéro de département<br>
E14TST : nombre total d'entreprises dans la ville<br>
E14TS0ND : nombre d'entreprises de taille inconnue ou nulle dans la ville<br>
E14TS1 : nombre d'entreprises de 1 à 5 employés dans la ville<br>
E14TS6 : nombre d'entreprises de 6 à 9 employés dans la ville<br>
E14TS10 : nombre d'entreprises de 10 à 19 employés dans la ville<br>
E14TS20 : nombre d'entreprises de 20 à 49 employés dans la ville<br>
E14TS50 : nombre d'entreprises de 50 à 99 employés dans la ville<br>
E14TS100 :  nombre d'entreprises de 100 à 199 employés dans la ville<br>
E14TS200 : nombre d'entreprises de 200 à 499 employés dans la ville<br>
E14TS500 : nombre d'entreprises de plus de 500 employés dans <br>




**Concernant les NaN**<br>
aucun NaN

**Concernant les Dtype**<br>
Dtype de la variable DEP doit sans doute être modifié en objet <br>

**Concernant la variable CODEGEO LIBGEO**

CODEGEO = code insee de la commune
LIBEGEO = intitulé de la commune 

si nécessaire les informations sur le **Code officiel géographique au 1er janvier 2022** est ici :
https://www.insee.fr/fr/information/6051727

différents fichiers y sont proposés avec différents niveaux : commune, canton, départements, régions.

le fichier pour les régions est ici : 
https://www.insee.fr/fr/statistiques/fichier/6051727/region_2022.csv

In [82]:
#rappport d'exploration

rap=pd.DataFrame() #création du rapport

rap['Colonne']=df.columns  # création de l'index

rename_axis=('N°Col')

rap['Dtype']=[df[i].dtype for i in df.columns ] # création de la colonne dtype

rap['T_NaN']=[(df[i].isna().sum()/df.shape[0]).round(1) for i in df.columns] # création de la colonne taux de NaN 

# création de la colonne max
max=[]
for i in df.columns :
  if df[i].dtype==object:
    max.append('nd')
  else :
    max.append(df[i].max())

rap['Max']=pd.DataFrame(max)

# création del a colonne min
min=[]
for i in df.columns :
  if df[i].dtype==object:
    min.append('nd')
  else :
    min.append(df[i].min())
rap['Min']=pd.DataFrame(min)

# création de la colonne description 

descript=['ID géographique de la ville',
          'nom de la ville',
          'numéro de région',
          'numéro de département',
          "nombre total d'entreprises dans la ville",
          "nombre d'entreprises de taille inconnue ou nulle dans la ville",
          "nombre d'entreprises de 1 à 5 employés dans la ville",
          "nombre d'entreprises de 6 à 9 employés dans la ville",
          "nombre d'entreprises de 10 à 19 employés dans la ville",
          "nombre d'entreprises de 20 à 49 employés dans la ville",
          "nombre d'entreprises de 50 à 99 employés dans la ville",
          "nombre d'entreprises de 100 à 199 employés dans la ville",
          "nombre d'entreprises de 200 à 499 employés dans la ville",
          "nombre d'entreprises de plus de 500 employés dans la ville"
          ]
rap['Description']=descript


# changement du Dtype de REG

df.REG=df.REG.astype('str')

# nombre de valeurs pour les variables catégorielles
distrib=[]

for i in df.columns :
    try :
      pd.to_numeric(df[i],errors='raise')
      distrib.append('nd')
    except ValueError :
      distrib.append(len(df[i].unique()))


rap['Distribution']=distrib 

display(rap)

len(df.REG.unique())


,Colonne,Dtype,T_NaN,Max,Min,Description,Distribution
0,CODGEO,object,0.0,nd,nd,ID géographique de la ville,36681
1,LIBGEO,object,0.0,nd,nd,nom de la ville,34142
2,REG,object,0.0,nd,nd,numéro de région,nd
3,DEP,object,0.0,nd,nd,numéro de département,101
4,E14TST,int64,0.0,427385,0,nombre total d'entreprises dans la ville,nd
5,E14TS0ND,int64,0.0,316603,0,nombre d'entreprises de taille inconnue ou nul...,nd
6,E14TS1,int64,0.0,76368,0,nombre d'entreprises de 1 à 5 employés dans la...,nd
7,E14TS6,int64,0.0,14836,0,nombre d'entreprises de 6 à 9 employés dans la...,nd
8,E14TS10,int64,0.0,10829,0,nombre d'entreprises de 10 à 19 employés dans ...,nd
9,E14TS20,int64,0.0,5643,0,nombre d'entreprises de 20 à 49 employés dans ...,nd


27

**ANALYSES COMPLEMENTAIRES**

Distribution des variables int hors

**Création de la colonne région**

nom de la région


In [58]:
# création du df régions à partir du Code officiel géographique au 1er janvier 2022
df_reg_source=pd.read_csv('https://www.insee.fr/fr/statistiques/fichier/6051727/region_2022.csv')
df_reg_source.head()

,REG,CHEFLIEU,TNCC,NCC,NCCENR,LIBELLE
0,1,97105,3,GUADELOUPE,Guadeloupe,Guadeloupe
1,2,97209,3,MARTINIQUE,Martinique,Martinique
2,3,97302,3,GUYANE,Guyane,Guyane
3,4,97411,0,LA REUNION,La Réunion,La Réunion
4,6,97608,0,MAYOTTE,Mayotte,Mayotte


In [59]:
# création du df région 

df_reg=df_reg_source[['REG','LIBELLE']]
df_reg.LIBELLE.unique()

array(['Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte',
       'Île-de-France', 'Centre-Val de Loire', 'Bourgogne-Franche-Comté',
       'Normandie', 'Hauts-de-France', 'Grand Est', 'Pays de la Loire',
       'Bretagne', 'Nouvelle-Aquitaine', 'Occitanie',
       'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur", 'Corse'],
      dtype=object)

In [60]:
# création de la colonne REG_NOM 
df=df.merge(right=df_reg, on='REG')
df=df.rename({'LIBELLE':'REG_NOM'},axis=1)
df.head()


,CODGEO,LIBGEO,REG,DEP,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500,REG_NOM
0,04001,Aiglun,93,04,85,60,13,5,3,3,0,1,0,0,Provence-Alpes-Côte d'Azur
1,04004,Allemagne-en-Provence,93,04,46,38,8,0,0,0,0,0,0,0,Provence-Alpes-Côte d'Azur
2,04005,Allons,93,04,6,6,0,0,0,0,0,0,0,0,Provence-Alpes-Côte d'Azur
3,04006,Allos,93,04,287,210,63,8,5,0,0,1,0,0,Provence-Alpes-Côte d'Azur
4,04007,Angles,93,04,3,3,0,0,0,0,0,0,0,0,Provence-Alpes-Côte d'Azur


In [63]:

len(df.REG_NOM.unique())


11